# 4D In-situ XCT of Aerosol Filtration Data Analysis; Python Workflow
#### by Matthew Jones

### Introduction
About this dataset: We used 4D (3D + time) X-ray Computed Tomography (XCT) at the Diamond Light Source Sychrotron I13-2 beamline to image how a TiO2 aerosol deposited in the pores and on the walls of a filter. This data set allows us to observe the transition from Deep bed to Cake filtration. This data set has the potential to be integrated into image based simulations and verify models of aerosol filtration on porous media.

About this notebook: We will include the entire workflow that takes reconstructed data through to a segmented 4D (3D + time) data set. We first segment the zeroth (i.e. clean volume) into background and filter segments. The filter segment mask is applied to the subsequent volumes which have all been registered onto the zeroth volume. These registered and masked subseqent volumes are then segmented (i.e. the aerosol deposits are segmented). See steps below;
1. Clean Z-axis banding issue in data
2. Binarisation of the zeroth volume
3. Registration of the subsequent volumes onto the zeroth volume
4. Rescaling the data
5. Non-local means filtering of the entire set
6. Apply Global Threshold
7. Visualisation

### Step 1; Clean Z-axis banding issue in reconstructed data

We have a banding issue in the data set where the gray level of the background varies from z-slice to z-slice. We will rescale the data to remove the noise using the method below. This method defines the global maximum of a reference image histogram (usually the whole volume reshaped to 2D image) and then resamples the input image so that it's histogram max matches that of the reference.

In [ ]:
import imageio as io
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import imageio as io
import numpy as np

vol = io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B0X.tif')
refimage = vol.reshape(-1)
image = vol[2105]

def equalise_globalmax_1(image, refimage):
    refhist = np.histogram(refimage.astype('Float32'), bins=range(256))
    imghist = np.histogram(image.astype('Float32'), bins=range(256))
    diff = refhist[0].tolist().index(max(refhist[0])) 
    - imghist[0].tolist().index(max(imghist[0]))                        
    resampled = image + diff
    return refhist, imghist, diff, resampled;

fig, axes = plt.subplots(ncols=1, nrows=3, figsize= (7, 7), sharex=True, sharey=False)
ax = axes.ravel()

refhist, imghist, diff, resampled = equalise_globalmax_1(image, refimage);

#ax[0].imshow(refimage, cmap=plt.cm.gray)
#ax[0].set_title('Reference')
#ax[0].axis('off')

ax[0].set_title('Reference Volume')
ax[0].hist(refimage.ravel(), bins=256)
ax[0].axvline(refhist[0].tolist().index(max(refhist[0])), color='r')
   
#ax[2].imshow(image, cmap=plt.cm.gray)
#ax[2].set_title('Original')
#ax[2].axis('off')
   
ax[1].set_title('Zth Slice')
ax[1].hist(image.ravel(), bins=256)
ax[1].axvline(imghist[0].tolist().index(max(imghist[0])), color='g')

#ax[4].imshow(resampled, cmap=plt.cm.gray)
#ax[4].set_title('Resampled')
#ax[4].axis('off')

ax[2].set_title('Resampled Slice to Reference')
ax[2].hist(resampled.ravel(), bins=256)
ax[2].axvline(refhist[0].tolist().index(max(refhist[0])), color='r')
ax[2].axvline(imghist[0].tolist().index(max(imghist[0])), color='g')

plt.show()

In [ ]:
#okay now we will loop z-slice through the function to process the entire volume
refhist = np.histogram(refimage.astype('Float32'), bins=range(256))
global_max = refhist[0].tolist().index(max(refhist[0]))

def equalise_globalmax_2(image, global_max):
    imghist = np.histogram(image.astype('Float32'), bins=range(256))
    diff = global_max - imghist[0].tolist().index(max(imghist[0]))                        
    resampled = image + diff
    return resampled;

for z in range(2110):
    vol[z] = equalise_globalmax_2(vol[z], global_max)

io.volwrite('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B0X.tif', vol, format='tiff')

In [ ]:
#that worked nicely so now we apply this to the whole data set
for y in range(1, 17):
    vol = io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d.tif'%(y))
    refimage = vol.reshape(-1)
    refhist = np.histogram(refimage.astype('Float32'), bins=range(256))
    global_max = refhist[0].tolist().index(max(refhist[0]))
    for z in range(2110):
        vol[z] = equalise_globalmax_2(vol[z], global_max)
    io.volwrite('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d.tif'%(y), vol, format='tiff')

In [ ]:
#see result of this resampling operation in the image of excel plot below.
plot_img = io.imread('E:/paper_2/resample_noise.png')
plt.imshow(plot_img)
plt.axis('off')

### Step 2; Binarisation of the Zeroth (i.e. clean unloaded) Volume

As described in the introduction we now binarise (segment into filter & background) the zeroth volume. This is the volume that was aquired before the aerosol was flowed through the filter. Thus we can segment this filter into two binary segments only. Since there is significant grayscale overlap between the deposits and filter in the subsequent 4D volumes we will apply this zeroth mask to those volumes so that the deposits can be segmented accurately. 
We apply a watershed segmentation, including the boundary in mask, and a highpass filter for high intensity pixels missed by the watershed.

In [ ]:
#Imports 
import numpy as np
import matplotlib.pyplot as plt
import imageio as io
from skimage import data, img_as_float
from skimage.io import imread_collection
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import crop
from skimage.morphology import binary_erosion, binary_dilation, cube
from skimage import filters
from skimage.filters import gaussian
import cv2

#Read in data for zeroth volume. This will be used as mask for the filter/cat phase.
vol_0 = img_as_float(crop(io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B0X.tif'), ((0, 0),(0, 0), (0, 0)), copy=True))

#2d wise non local means
sigma_est = np.mean(estimate_sigma(vol_0, multichannel=False))
for z in range(0, 2110):
    vol_0[z] = denoise_nl_means(vol_0[z], h=4*sigma_est, fast_mode=True,
                                  patch_size=9, patch_distance=5, multichannel=False)
vol_0 = vol_0 * 255

#Gaussian Filter
vol_0 = gaussian(vol_0, sigma=0.20, output=None, mode='nearest', cval=0, multichannel=False, preserve_range=True, 
                        truncate=4.0)

for y in range(0, 2110):
    io.imwrite('E:/paper_2/Volumes_Bare/NLM0_TiffSaver/TiffSaver_%d.tif'%(y),
               vol_0[y].astype(np.uint8), format='tiff')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio as io
from skimage import filters

kernel = np.ones((2, 2),np.uint8)

for z in range(0, 2110):
    #Apply binary threshold 1
    img = cv2.imread('E:/paper_2/Volumes_Bare/NLM0_TiffSaver/TiffSaver_%d.tif'%(z))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    thresh_yen = filters.threshold_yen(img.astype('uint8'))
    ret, thresh = cv2.threshold(gray,thresh_yen+7,255,cv2.THRESH_BINARY)
    
    #Erosion/Dilation of binary 1 and defining 'sure foreground & background' for watershed labels
    #sure foreground
    sure_fg = cv2.erode(thresh,kernel,iterations=2)
    sure_fg = sure_fg.astype(np.uint8)
    
    # sure background area
    sure_bg = cv2.dilate(thresh,kernel,iterations=2)
    sure_bg = sure_bg.astype(np.uint8)
    
    # Finding unknown region
    unknown = cv2.subtract(sure_bg,sure_fg)
    
    #Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    
    #Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    
    #Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    
    #best solution was watershed (inc. boundary pixels) + a high pass threshold for high intensity pixels missed by watershed
    markers = cv2.watershed(img,markers)
    markers_binary = markers > 1
    unknown = markers < 1
    markers_binary_unknown = markers_binary + unknown
    masked = gray * (1 - markers_binary_unknown)
    masked_highpass = masked > thresh_yen + 29
    final_mask = markers_binary_unknown + masked_highpass
    
    #save slice
    io.imwrite('E:/paper_2/Volumes_Bare/Mask_TiffSaver/TiffSaver_%d.tif'%(z), final_mask.astype(np.uint8), format='tiff')
    
#Plot images illustrating process
fig, axes = plt.subplots(nrows=9, ncols=1, figsize = (50, 50), 
                         sharex=True, sharey=True)
axes[0].imshow(img, cmap = 'gray')
axes[1].imshow(thresh, cmap = 'gray')
axes[2].imshow(sure_fg, cmap = 'gray')
axes[3].imshow(sure_bg, cmap='gray')
axes[4].imshow(markers_binary, cmap='gray')
axes[5].imshow(markers_binary_unknown, cmap='gray')
axes[6].imshow(masked, cmap='gray')
axes[7].imshow(masked_highpass, cmap='gray')
axes[8].imshow(final_mask, cmap='gray')

**IMPORTANT NOTE:** This mask was loaded into Avizo so that **labels smoothing** could be applied with value 3 in Z, 2 in Y, 2 in X. Furthermore it is important to note that there may be some streaking on the wall surface of the left hand wall. Check this before using volumes for analysis/simulation.

### Step 3; Registration of subsequent 4D volumes onto the zeroth volume
Since we are going to apply the zeroth filter mask to all subsequent volumes in the data set it is important that the volumes are accuretly registered onto eachover (i.e. volumes are aligned). Otherwise this will mean that filter pixels are included in later deposit segments. Here we use the Simple ITK library to register volumes.

In [ ]:
#we will now register sebsequent volumes in the time series onto the binary volume from part 2
#it is critical the registration is good as initial binary will be aplied to subsequent volumes

from PIL import Image, ImageChops
import math
import numpy as np
import imageio as io
from skimage import img_as_float
from skimage.metrics import mean_squared_error as mse
from skimage.util import crop

#first we need to make some 'registration volumes' from the full volume so that sitk can
#read vol w/o mem err
for s in range(1, 17):
    load = io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d.tif'%(s))
    crop = load[102:498, 302:498, 1002:1098]
    io.volwrite('E:/paper_2/Volumes_Bare/Registration_vol_bare/TiffSaver-reg_B%d.tif'%(s),
                crop, format='tiff')

load = io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B0X.tif')
crop = load[102:498, 302:498, 1002:1098]
io.volwrite('E:/paper_2/Volumes_Bare/Registration_vol_bare/TiffSaver-reg_B0X.tif',
            crop, format='tiff')

In [ ]:
#function that calculates the root mean square error between im1 and im2
def rmsdiff(im1, im2):
    """Calculates the root mean square error (RSME) between two images"""
    return math.sqrt(mse(im1, im2))

#Loads and crops im1 and im2 for rmsdiff function
#loops im2 loader through t dimension in data series
im1 = io.volread('E:/paper_2/Volumes_Bare/Registration_vol_bare/TiffSaver-reg_B0X.tif')
.astype('Float32')
for x in range(1, 17):
    im2 = io.volread('E:/paper_2/Volumes_Bare/Registration_vol_bare/TiffSaver-reg_B%d.tif'%(x))
    .astype('Float32')
    print(f"RMS of tomo %d = {rmsdiff(im1[2:398, 2:198, 2:98], im2[2:398, 2:198, 2:98])}"%(x))

In [ ]:
#to check we have analysed correct shape/image region
import matplotlib.pyplot as plt
print(im1[2:398, 2:198, 2:98].shape)
plt.imshow(im1[200, 2:198, 2:98], cmap = 'gray')

In [ ]:
#that'll do
from __future__ import print_function
import SimpleITK as sitk
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed

#the dimension of the image and the origin
dimension = 3
point = (1.0, 1.0, 1.0)
#this function will print where the origin will be translated to 
def transform_point(transform, point):
    transformed_point = transform.TransformPoint(point)
    print('Point ' + str(point) + ' transformed is ' + str(transformed_point))
#this function will visualises
def myshow(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayViewFromImage(img)
    spacing = img.GetSpacing()    
    ysize = nda.shape[0]
    xsize = nda.shape[1]  
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi
    fig = plt.figure(title, figsize=figsize, dpi=dpi)
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    extent = (0, xsize*spacing[1], 0, ysize*spacing[0])
    t = ax.imshow(nda,
            extent=extent,
            interpolation='hamming',
            cmap='gray',
            origin='lower')
    if(title):
        plt.title(title)

#this function executes the resampling
def resample(image, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    reference_image = image
    interpolator = sitk.sitkNearestNeighbor
    default_value = 0
    return sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)
moving_image = sitk.ReadImage("E:/paper_2/Volumes_Bare/Registration_vol_bare/TiffSaver-reg_B1.tif",
                              sitk.sitkFloat32)
moving_image_cropped = moving_image[:, :, :] 
translation = sitk.TranslationTransform(dimension)
translation.SetParameters((0.0, 0.0, 0.0))
translation.SetOffset((0.0, 0.0, 0.0))
transform_point(translation, point)
resampled = resample(moving_image_cropped, translation)

In [ ]:
fixed_image = sitk.ReadImage("E:/paper_2/Volumes_Bare/Registration_vol_bare/TiffSaver-reg_B0X.tif",
                             sitk.sitkFloat32)
fixed_image_cropped = fixed_image[:, :, :]
translation_0 = sitk.TranslationTransform(dimension)
translation_0.SetParameters((0.0, 0.0, 0.0))
translation_0.SetOffset((0.0, 0.0, 0.0))
transform_point(translation_0, point)
resampled_0 = resample(fixed_image_cropped, translation_0)

In [ ]:
RMS0 = (sitk.GetArrayFromImage(resampled_0[2:98, 2:198, 2:398]))
RMS1 = (sitk.GetArrayFromImage(resampled[2:98, 2:198, 2:398]))
print(rmsdiff(RMS0, RMS1)) #this has to equal the value originally calculated up top or something is off

In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image_cropped, 
                                                      moving_image_cropped, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMeanSquares()
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkNearestNeighbor)  
#This is the least computationally demanding interpolator 
registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,0,0,20,10,10], stepLength = 1) 
registration_method.SetOptimizerScales([1,1,1,1,1,1])

#Perform the registration in-place so that the initial_transform is modified.
registration_method.SetInitialTransform(initial_transform, inPlace=True)
registration_method.Execute(fixed_image_cropped, moving_image_cropped)

print('course initial transform is: ' + str(initial_transform.GetParameters()))

In [ ]:
translation_test = sitk.TranslationTransform(dimension)
translation_test.SetParameters((1.0, -3.0, 3.0)) #put the best transform from above here!!!
translation.SetOffset((1.0, -3.0, 3.0)) ##put the best transform from above here!!!
transform_point(translation_test, point)
resampled_test = resample(moving_image_cropped, translation_test)
myshow(resampled_test[10, :, :], 'Resampled Translation')

In [ ]:
RMS0_test = (sitk.GetArrayFromImage(resampled_0[2:98, 2:198, 2:398])).astype('uint8')
RMS1_test = (sitk.GetArrayFromImage(resampled_test[2:98, 2:198, 2:398])).astype('uint8')
print(rmsdiff(RMS0_test, RMS1_test))

In [ ]:
# we have shown that this method improves the registration of the subsequent grayscale images 
#onto the binary i.e because the RMSE is less after the translation
# now we loop the volumes through the above code in order to apply to the whole set
for x in range(1, 17):
    initial_transform = sitk.CenteredTransformInitializer(fixed_image_cropped, 
                                                      moving_image_cropped, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMeanSquares()
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkNearestNeighbor)  
    #This is the least computationally demanding interpolator 
    registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,0,0,20,10,10], stepLength = 1) 
    registration_method.SetOptimizerScales([1,1,1,1,1,1])
    moving_image = sitk.ReadImage("E:/paper_2/Volumes_Bare/Registration_vol_bare/TiffSaver-reg_B%d.tif"%(x)
                                  , sitk.sitkFloat32)
    moving_image_cropped = moving_image[:, :, :]
    registration_method.SetInitialTransform(initial_transform, inPlace=True)
    registration_method.Execute(fixed_image_cropped, moving_image_cropped)
    print('best transformation %d: '%(x) + str(initial_transform.GetParameters()))

In [ ]:
#Ok! Let's now apply the translations to the data set

from __future__ import print_function
import SimpleITK as sitk
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed

dimension = 3
def resample(image, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    reference_image = image
    interpolator = sitk.sitkNearestNeighbor
    default_value = 0
    return sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)

#had to crop and save some smaller volumes to stop mem errors
#import imageio as io
#for x in range(1, 17):
    #imga = io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d.tif'%(x))[:1055, :, :]
    #imgb = io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d.tif'%(x))[1055:, :, :]
    #io.volwrite('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d_a.tif', imga, format='tiff')
    #io.volwrite('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d_b.tif', imgb, format='tiff')

#make lists
#these params where calculated above from the registration process but with the inital transform 
#added into the params this initial transform is from the cropping process and not the
#initial tranform from the reg above

param_1 = (1.0, -3.0, 3.0)
param_2 = (1.0, -2.0, 3.0)
param_3 = (1.0, -2.0, 3.0)
param_4 = (1.0, -3.0, 3.0)
param_5 = (1.0, -3.0, 3.0)
param_6 = (0.0, -3.0, 3.0)
param_7 = (0.0, -3.0, 3.0)
param_8 = (0.0, -3.0, 3.0)
param_9 = (1.0, -3.0, 3.0)
param_10 = (1.0, -3.0, 3.0)
param_11 = (0.0, -3.0, 4.0)
param_12 = (0.0, -3.0, 4.0)
param_13 = (0.0, -3.0, 4.0)
param_14 = (0.0, -3.0, 4.0)
param_15 = (0.0, -3.0, 4.0)
param_16 = (0.0, -3.0, 4.0)

parameters = [param_1, param_2, param_3, param_4, param_5, param_6, param_7, param_8,
              param_9, param_10, param_11, param_12, param_13, param_14, param_15, param_16]
resampled_list = ['r_1', 'r_2', 'r_3', 'r_4', 'r_5', 'r_6', 'r_7', 'r_8', 'r_9', 'r_10',
                  'r_11', 'r_12', 'r_13', 'r_14', 'r_15', 'r_16']

for x in range(9, 16):
    image = sitk.ReadImage("E:/paper_2/Volumes_Bare/TiffSaver-tomo_B%d_b.tif"%(x+1), sitk.sitkFloat32)
    translation = sitk.TranslationTransform(dimension)
    translation.SetParameters(parameters[x])
    translation.SetOffset(parameters[x])
    resampled_list[x] = resample(image, translation)

In [ ]:
import imageio as io
#now save these for future segmentation
for z in range(9, 16):
    io.volsave('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_b.tif'%(z+1),  
               sitk.GetArrayFromImage(resampled_list[z]).astype(np.uint8), format='tiff')

In [ ]:
#in sitk it is possible to do even finer sub pixel translations because the image is 
#considered to bound a physical space rather than discrete pixel locations
#we will now apply a finer registration

moving_image = sitk.ReadImage('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B1_a.tif'
                              , sitk.sitkFloat32)
moving_image_cropped = moving_image[1010:1090, 310:490, 110:490]
fixed_image = sitk.ReadImage('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B0X.tif', sitk.sitkFloat32)
fixed_image_cropped = fixed_image[1010:1090, 310:490, 110:490]
    
for l in range(1, 17):
    initial_transform = sitk.CenteredTransformInitializer(fixed_image_cropped, 
                                                      moving_image_cropped, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMeanSquares()
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkNearestNeighbor)  
    #This is the least computationally demanding interpolator 
    registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,0,0,10,10,10], stepLength = 0.1) 
    registration_method.SetOptimizerScales([1,1,1,1,1,1])
    moving_image = sitk.ReadImage('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(l)
                                  , sitk.sitkFloat32)
    moving_image_cropped = moving_image[1010:1090, 310:490, 110:490]
    registration_method.SetInitialTransform(initial_transform, inPlace=True)
    registration_method.Execute(fixed_image_cropped, moving_image_cropped)
    print('best transformation %d: '%(l) + str(initial_transform.GetParameters()))

In [ ]:
#Ok! Let's now apply the translations to the data set

from __future__ import print_function
import SimpleITK as sitk
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed

dimension = 3
def resample(image, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    reference_image = image
    interpolator = sitk.sitkNearestNeighbor
    default_value = 0
    return sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)

params_1 = (0.0, 0.0, 0.0)
params_2 = (0.0, 0.0, 0.0)
params_3 = (0.0, 0.0, 0.0)
params_4 = (0.0, 0.0, 0.0)
params_5 = (-0.9, 0.0, 0.0)
params_6 = (0.0, 0.0, 0.0)
params_7 = (0.1, 0.0, 0.0)
params_8 = (0.6000000000000001, 0.0, 0.0)
params_9 = (0.0, 0.0, 0.0)
params_10 = (0.0, 0.0, 0.0)
params_11 = (0.0, 0.0, 0.0)
params_12 = (0.0, 0.0, 0.0)
params_13 = (0.0, 0.0, 0.0)
params_14 = (0.0, 0.0, 0.0)
params_15 = (0.0, 0.0, 0.0)
params_16 = (0.0, 0.0, 0.0)

parameters_1 = [params_1, params_2, params_3, params_4, params_5, params_6, params_7, params_8,
              params_9, params_10, params_11, params_12, params_13, params_14, params_15, params_16]
resampled_list_1 = ['r_1', 'r_2', 'r_3', 'r_4', 'r_5', 'r_6', 'r_7', 'r_8', 'r_9', 'r_10', 'r_11', 'r_12',
                  'r_13', 'r_14', 'r_15', 'r_16']
for x in range(9, 16):
    image = sitk.ReadImage('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_b.tif'%(x+1)
                           , sitk.sitkFloat32)
    translation = sitk.TranslationTransform(dimension)
    translation.SetParameters(parameters_1[x])
    translation.SetOffset(parameters_1[x])
    resampled_list_1[x] = resample(image, translation)

In [ ]:
import imageio as io
for x in range(9, 16):
    io.volsave('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_b.tif'%(x+1), 
               sitk.GetArrayFromImage(resampled_list_1[x]).astype(np.uint8), format='tiff')

### Step 4; Rescaling the Data 4D
The average and background gray level move around from 4D volume to 4D volume. In order to be able to apply a global threshold to all the set we want pixels to all be in the same relative range. Thus we rescale, using a simialar method to step 1. 

In [ ]:
#here we will rescale all volumes so that the global maximum is 
#at the same gray intensity across the whole set
import matplotlib.pyplot as plt
import imageio as io
import numpy as np

def rescale_4D(image, rescale_to_value):
    imghist = np.histogram(image.astype('Float32'), bins=range(256))
    diff = rescale_to_value - imghist[0].tolist().index(max(imghist[0]))                        
    resampled = image + diff
    return resampled;

Ref_vol = io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B0X.tif')
refimage = Ref_vol.reshape(-1)
refhist = np.histogram(refimage.astype('Float32'), bins=range(256))
rescale_to_value = refhist[0].tolist().index(max(refhist[0]))

for x in range(1, 17):
    image = io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(x))
    resampled = rescale_4D(image, rescale_to_value)
    io.volwrite('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(x),
                resampled, format='tiff')

### Step 5; Apply Non-local Means Filter to Whole Set
In-order to segment the deposit phase in the next steps we need to smooth the image so that noise does not effect the segmentation. Here we apply the same nlm parameters as used in step 2 to the whole data set. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio as io
from skimage import data, img_as_float
from skimage.io import imread_collection
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import crop
from skimage.morphology import binary_erosion, binary_dilation, cube
from skimage import filters
from skimage.filters import gaussian

vol_0 = img_as_float(io.volread('E:/paper_2/Volumes_Bare/TiffSaver-tomo_B0X.tif'))
sigma_est_B0X = np.mean((estimate_sigma(vol_0, multichannel=False)))
for z in range(2110):
    vol_0[z] = denoise_nl_means(vol_0[z], h=4*sigma_est_B0X, fast_mode=True,
                                patch_size=9, patch_distance=5, multichannel=False)
vol_0_nlm = vol_0 * 255    
vol_0_gaussian = gaussian(vol_0_nlm, sigma=0.20, output=None, mode='nearest', cval=0, 
                          multichannel=False, preserve_range=True, truncate=4.0)
for w in range(2110):
    io.imwrite('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B0X/TiffSaver-tomo_B0X_%d.tif'%(w),
               vol_0_gaussian[w].astype(np.uint8), format='tiff')

In [ ]:
#2d wise non local means & gaussian of whole 4d set
for l in range(2, 17):
    vol_0 = img_as_float
    (io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(l)))
    sigma_est_a = np.mean(estimate_sigma(vol_0, multichannel=False))
    for z in range(1055):
        vol_0[z] = denoise_nl_means(vol_0[z], h=4*sigma_est_a, fast_mode=True, 
                                    patch_size=9, patch_distance=5, multichannel=False)
    vol_0_nlm = vol_0 * 255
    vol_0_gaussian = gaussian(vol_0_nlm, sigma=0.20, output=None, mode='nearest', cval=0,
                                  multichannel=False, preserve_range=True, truncate=4.0)
    io.volwrite('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(l), 
                vol_0_gaussian.astype(np.uint8), format='tiff')

for l in range(1, 17):
    vol_0 = img_as_float
    (io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_b.tif'%(l)))
    sigma_est_b = np.mean(estimate_sigma(vol_0, multichannel=False))
    for z in range(1055):
        vol_0[z] = denoise_nl_means(vol_0[z], h=4*sigma_est_b, fast_mode=True, 
                                    patch_size=9, patch_distance=5, multichannel=False)
    vol_0_nlm = vol_0 * 255
    vol_0_gaussian = gaussian(vol_0_nlm, sigma=0.20, output=None, mode='nearest', cval=0,
                                  multichannel=False, preserve_range=True, truncate=4.0)
    io.volwrite('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_b.tif'%(l), 
                vol_0_gaussian.astype(np.uint8), format='tiff')

### Step 6; Apply global threshold and save segmented volumes
Now that the data has been properly clean and organised it is possible to segment using a global threshold across the entire 4D data set. Various volumes are saved for later analysis.


In [ ]:
import imageio as io
import numpy as np
from skimage import morphology
from skimage.morphology import binary_erosion, binary_dilation, cube
from skimage import filters

#we are going to apply a global TiO2 deposit threshold across
#the entire 4D data set and create segmented labelled volumes
filter_seg_a = io.volread('E:/paper_2/Volumes_Bare/Mask_TiffSaver_a.tif')
filter_seg_b = io.volread('E:/paper_2/Volumes_Bare/Mask_TiffSaver_b.tif')
channel_mask_a = io.volread('E:/paper_2/Volumes_Bare/Channel_Mask_TiffSaver.tif')[:1055, :, :]
channel_mask_b = io.volread('E:/paper_2/Volumes_Bare/Channel_Mask_TiffSaver.tif')[1055:, :, :]

#we are going to create an image of the gray pixels of the filter that are missed by the filter mask 
#this will be subtrated from the 4d in-situ volumes so that they are not included in the deposit segment
clean_vol = io.volread('E:/paper_2/Volumes_Bare/NLM0_TiffSaver.tif')
for m in range(0, 1055):
    clean = clean_vol[m]
    thresh0 = filters.threshold_yen(clean.astype('uint8'))
    missed_mask = clean * (1-filter_seg_a[m]) > thresh0
    missed = clean * missed_mask
    clean_vol[m] = missed
for m in range(0, 1055):
    clean = clean_vol[m+1055]
    thresh0 = filters.threshold_yen(clean.astype('uint8'))
    missed_mask = clean * (1-filter_seg_b[m]) > thresh0
    missed = clean * missed_mask
    clean_vol[m+1055] = missed

In [ ]:
from skimage import morphology
channel_only_mask_a = io.volread('E:/paper_2/Volumes_Bare/channel_only_mask_a.tif')
channel_only_mask_b = io.volread('E:/paper_2/Volumes_Bare/channel_only_mask_b.tif')
for x in range(1, 17):
    vol = io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(x))
    vol = vol - 0.7 * clean_vol[:1055, :, :]
    refvol = vol * channel_only_mask_a #here we are finding the hist max (i.e. mode pixel intensity) and defining the thresh from this value
    refimage = refvol.reshape(-1)
    refhist = np.histogram(refimage.astype(np.float32), bins=range(256))
    refhist[0][0] = 0
    thresh_bed = refhist[0].tolist().index(np.amax(refhist[0])) + 3
    thresh_cake = refhist[0].tolist().index(np.amax(refhist[0])) + 5
    vol_TiO2_bed = (vol * (1 - filter_seg_a)) * channel_mask_a > thresh_bed
    vol_TiO2_bed = binary_erosion(vol_TiO2_bed, cube(2))
    vol_TiO2_bed = binary_dilation(vol_TiO2_bed, cube(3)) * (1 - filter_seg_a)
    vol_TiO2_cake = (vol * (1 - filter_seg_a)) * (1 - channel_mask_a) > thresh_cake
    vol_TiO2_cake = morphology.remove_small_objects(vol_TiO2_cake, 80, connectivity=2)
    vol_TiO2_cake = binary_erosion(vol_TiO2_cake, cube(3))
    vol_TiO2_cake = binary_dilation(vol_TiO2_cake, cube(4))
    vol_TiO2_cake = binary_erosion(binary_dilation(vol_TiO2_cake, cube(7)), cube(7))
    vol_TiO2 = vol_TiO2_cake + vol_TiO2_bed
    io.volwrite('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_%da.tif'%(x), vol_TiO2.astype(np.uint8), format='tiff')
    segmented = filter_seg_a + (2 * vol_TiO2) # background = 0, filter = 1, TiO2 = 2
    io.volwrite('E:/paper_2/Volumes_Bare/Segmented_vols/Segmented_vol_%da.tif'%(x), segmented.astype(np.uint8), format='tiff')
for x in range(5, 6):
    vol = io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_b.tif'%(x))
    vol = vol - 0.7 * clean_vol[1055:, :, :]
    refvol = vol * channel_only_mask_b
    refimage = refvol.reshape(-1)
    refhist = np.histogram(refimage.astype(np.float32), bins=range(256))
    refhist[0][0] = 0
    thresh_bed = refhist[0].tolist().index(np.amax(refhist[0])) + 3
    thresh_cake = refhist[0].tolist().index(np.amax(refhist[0])) + 5
    vol_TiO2_bed = (vol * (1 - filter_seg_b)) * channel_mask_b > thresh_bed
    vol_TiO2_bed = binary_erosion(vol_TiO2_bed, cube(2))
    vol_TiO2_bed = binary_dilation(vol_TiO2_bed, cube(3)) * (1 - filter_seg_a)
    vol_TiO2_cake = (vol * (1 - filter_seg_b)) * (1 - channel_mask_b) > thresh_cake
    vol_TiO2_cake = morphology.remove_small_objects(vol_TiO2_cake, 80, connectivity=2)
    vol_TiO2_cake = binary_erosion(vol_TiO2_cake, cube(3))
    vol_TiO2_cake = binary_dilation(vol_TiO2_cake, cube(4))
    vol_TiO2_cake = binary_erosion(binary_dilation(vol_TiO2_cake, cube(7)), cube(7))
    vol_TiO2 = vol_TiO2_cake + vol_TiO2_bed
    io.volwrite('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_%db.tif'%(x), vol_TiO2.astype(np.uint8), format='tiff')
    segmented = filter_seg_b + (2 * vol_TiO2) # background = 0, filter = 1, TiO2 = 2
    io.volwrite('E:/paper_2/Volumes_Bare/Segmented_vols/Segmented_vol_%db.tif'%(x), segmented.astype(np.uint8), format='tiff')
    
#If needing 2D wise to correct for innacurate segmentation in specific slices 
#from skimage.morphology import binary_erosion, binary_dilation, square
#for x in range(10, 11):
    #vol = io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(x))
    #for y in range(0, 1055):
        #img = vol[y]
        #thresh = filters.threshold_yen(img.astype('uint8')) - 6
        #img = img - 0.7 * clean_vol[y]
        #img_TiO2_bed = (img * (1 - filter_seg_a[y])) * channel_mask_a[y] > thresh
        #img_TiO2_cake = (img * (1 - filter_seg_a[y])) * (1 - channel_mask_a[y]) > thresh
        #img_TiO2_cake = binary_erosion(img_TiO2_cake, square(3))
        #img_TiO2_cake = binary_dilation(img_TiO2_cake, square(5))
        #img_TiO2 = img_TiO2_cake + img_TiO2_bed
        #vol[y] = img_TiO2
    #io.volwrite('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_%da.tif'%(x), vol.astype(np.uint8), format='tiff')
    #segmented = filter_seg_a + (2 * vol) # background = 0, filter = 1, TiO2 = 2
    #io.volwrite('E:/paper_2/Volumes_Bare/Segmented_vols/Segmented_vol_%da.tif'%(x), segmented.astype(np.uint8), format='tiff')

In [ ]:
import imageio as io
import numpy as np

for x in range(1, 17):
    vol = io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_a.tif'%(x))
    vol_TiO2 = io.volread('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_%da.tif'%(x))
    TiO2_gray_vol = (vol * vol_TiO2)
    io.volwrite('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_gray_%da.tif'%(x), TiO2_gray_vol.astype(np.uint8), format='tiff')
    
for x in range(1, 17):
    vol = io.volread('E:/paper_2/Volumes_Bare/Registered_vols/TiffSaver-tomo_B%d_b.tif'%(x))
    vol_TiO2 = io.volread('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_%db.tif'%(x))
    TiO2_gray_vol = (vol * vol_TiO2)
    io.volwrite('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_gray_%db.tif'%(x), TiO2_gray_vol.astype(np.uint8), format='tiff')

### Step 7; Visualisation
The volumes from step 6 are loaded, cropped and saved for analysis in Avizo. In avizo the volumes are rendered. Below we show segmented rendering of the filter and background and intensity (physics colour map) of the TiO2 phase as they evolve in an animation.

In [ ]:
import imageio as io
import numpy as np
#create small 'visualisation' volumes 250x250x250 upper right channel wall
mask_a_vis = io.volread('E:/paper_2/Volumes_Bare/Mask_TiffSaver_a.tif')[0:250, 428:678, 852:1102]
mask_b_vis = io.volread('E:/paper_2/Volumes_Bare/Mask_TiffSaver_b.tif')[:250, 340:590, 834:1084]
io.volwrite('E:/paper_2/Volumes_Bare/Visualisation_vols/mask_a_vis.tif', mask_a_vis, format='tiff')
io.volwrite('E:/paper_2/Volumes_Bare/Visualisation_vols/mask_b_vis.tif', mask_b_vis, format='tiff')

for x in range(1, 17):
    TiO2_vol_gray_vis = io.volread('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_gray_%da.tif'%(x))[0:250, 428:678, 852:1102]
    TiO2_vol_vis = io.volread('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_%da.tif'%(x))[0:250, 428:678, 852:1102]
    io.volwrite('E:/paper_2/Volumes_Bare/Visualisation_vols/TiO2_vol_gray_%da.tif'%(x), TiO2_vol_gray_vis, format='tiff')
    io.volwrite('E:/paper_2/Volumes_Bare/Visualisation_vols/TiO2_vol_%da.tif'%(x), TiO2_vol_vis, format='tiff')

for x in range(1, 17):
    TiO2_vol_gray_vis = io.volread('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_gray_%db.tif'%(x))[:250, 340:590, 834:1084]
    TiO2_vol_vis = io.volread('E:/paper_2/Volumes_Bare/TiO2_vols/TiO2_vol_%db.tif'%(x))[:250, 340:590, 834:1084]
    io.volwrite('E:/paper_2/Volumes_Bare/Visualisation_vols/TiO2_vol_gray_%db.tif'%(x), TiO2_vol_gray_vis, format='tiff')
    io.volwrite('E:/paper_2/Volumes_Bare/Visualisation_vols/TiO2_vol_%db.tif'%(x), TiO2_vol_vis, format='tiff')

In [ ]:
import imageio as io
list = []
for z in range(0, 17):
    im = io.imread('E:/paper_2/Volumes_Bare/animation/avizo_snap_%da.jpg'%(z))
    list.append(im)
kargs = { 'duration': 2 }
imageio.mimsave('E:/paper_2/Volumes_Bare/animation/animation.gif', list, format='gif', duration = 1)

![SegmentLocal](animation.gif "segment")